Advanced Lane Lines
==

# The flow of pipeline
1. Undistort the image with camera calibration
1. Make binary image by threshold for color conversion and gradients
1. Warp the image to birds eye view

### Undistort the image with camera calibration

In [ ]:
import cv2
import pickle
with open('calib_cam_dist_pickle.p', 'rb') as f:
    cam_calib = pickle.load(f)

def undistort(src, calib):
    return cv2.undistort(src, calib['mtx'], calib['dist'], None, calib['mtx'])

### Make thresholded binary image for lane lines

In [ ]:
import numpy as np

def binarize_for_line(src, params):
    
    def sobel_thresh(gray, params):
        
        def mag_thresh(sobelx, sobely, thr_min, thr_max):
            # Calculate the gradient magnitude
            gradmag = np.sqrt(sobelx**2 + sobely**2)
            # Rescale to 8 bit
            scale_factor = np.max(gradmag)/255 
            gradmag = (gradmag/scale_factor).astype(np.uint8) 
            # Create a binary image of ones where threshold is met, zeros otherwise
            binary_mag = np.zeros_like(gradmag)
            binary_mag[(gradmag >= thr_min) & (gradmag <= thr_max)] = 1
            return binary_mag
        
        def dir_thresh(sobelx, sobely, thr_min, thr_max):
            # Take the absolute value of the gradient direction, 
            # apply a threshold, and create a binary image result
            absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
            binary_dir =  np.zeros_like(absgraddir)
            binary_dir[(absgraddir >= thr_min) & (absgraddir <= thr_max)] = 1
            return binary_dir
        
        gray = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)
        # Take both Sobel x and y gradients
        kernel_size = params['ksize']
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=kernel_size)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=kernel_size)
        mag_binary = mag_thresh(sobelx, sobely, *params['mag_thr'])
        dir_binary = dir_thresh(sobelx, sobely, *params['dir_thr'])
        mag_and_dir = np.zeros_like(mag_binary)
        mag_and_dir[(mag_binary == 1) & (dir_binary == 1)] = 1
        return mag_and_dir
    
    def color_thresh(src, thr_min, thr_max):
        hls = cv2.cvtColor(src, cv2.COLOR_RGB2HLS)
        s_channel = hls[:,:,2]
        s_binary = np.zeros_like(s_channel)
        s_binary[(s_channel >= thr_min) & (s_channel <= thr_max)] = 1
        return s_binary
    
    # Combine the two binary thresholds
    grad_binary = sobel_thresh(src, params['sobel'])
    color_binary = color_thresh(src, *params['color_thr'])
    combined_binary = np.zeros_like(grad_binary)
    combined_binary[(grad_binary == 1) & (color_binary == 1)] = 1
    return combined_binary
    
    
    

### Warp the image to birds eye view

In [ ]:
class Warper(object):
    def __init__(self, src, dst):
        self.M = cv2.getPerspectiveTransform(src, dst)
        self.M_inv = cv2.getPerspectiveTransform(dst, src)
        return
    
    def __call__(self, img, inverse=False):
        mtx = self.M if not inverse else self.M_inv
        img_size = (img.shape[1], img.shape[0])
        warped = cv2.warpPerspective(img, mtx, img_size)
        return warped


class LaneWarper():
    def __init__(self, w, h):
        self.src = np.float32(
            [[(w / 2) - 55, h / 2 + 100],
             [((w / 6) - 10), h],
             [(w * 5 / 6) + 60, h],
             [(w / 2 + 55), h / 2 + 100]])

        self.dst = np.float32(
            [[(w / 4), 0],
             [(w / 4), h],
             [(w * 3 / 4), h],
             [(w * 3 / 4), 0]])
        self.warper = Warper(self.src, self.dst)
        return
    
    def __call__(self, img):
        return self.warper(img)


IMG_WIDTH = 1280
IMG_HEIGHT = 720
lane_warper = LaneWarper(IMG_WIDTH, IMG_HEIGHT)

In [ ]:
def show_warper_example():
    %matplotlib inline
    import matplotlib.pyplot as plt

    src = cv2.imread('test_images/straight_lines1.jpg')
    undistort_img = undistort(src, cam_calib)
    
    # Draw src of warp M in red
    pts = lane_warper.src.astype(np.int32)
    pts = pts.reshape((-1,1,2))
    undistort_img = cv2.polylines(undistort_img, [pts], True, (0, 0, 255), thickness=2)
    
    dst = lane_warper(undistort_img)

    # Draw dst of warp M in blue
    pts = lane_warper.dst.astype(np.int32)
    pts = pts.reshape((-1,1,2))
    dst= cv2.polylines(dst, [pts], True, (255, 0, 0), thickness=3)
    
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    ax1.imshow(cv2.cvtColor(undistort_img, cv2.COLOR_BGR2RGB))
    ax1.set_title('Undistort Image', fontsize=30)
    ax2.imshow(cv2.cvtColor(dst, cv2.COLOR_BGR2RGB))
    ax2.set_title('Warped Image', fontsize=30)
    return

show_warper_example()

###  Identify lane-line pixels and fit their positions with a polynomial

In [ ]:
VISUALIZE = False


class LaneFinder(object):
    def __init__(self, params):
        # HYPERPARAMETERS
        # Choose the number of sliding windows
        self.nwindows = params.get('nwindows', 9)
        # Set the width of the windows +/- margin
        self.margin = params.get('margin', 100)
        # Set minimum number of pixels found to recenter window
        self.minpix = params.get('minpix', 50)
        # Set max number of pixels for rediscovering the lane pixels with sliding window search
        self.rediscover_thr = params.get('rediscover_thr', 50)

        # Polynomial fit values from the previous frame
        self.left_fit = None
        self.right_fit = None
        # self.left_fit = np.array([ 2.13935315e-04, -3.77507980e-01,  4.76902175e+02])
        # self.right_fit = np.array([4.17622148e-04, -4.93848953e-01,  1.11806170e+03])
        return

    def process(self, binary_warped):
        if self.left_fit is None or self.right_fit is None:
            # Search with sliding window at the fist time
            leftx, lefty, rightx, righty = self.find_lane_pixels(binary_warped)
        else:
            # Search in a margin around the previous line position,
            leftx, lefty, rightx, righty = self.search_around_poly(binary_warped)
            if len(leftx) < self.rediscover_thr or len(rightx) < self.rediscover_thr:
                # Search again with sliding window when losing track of lines
                leftx, lefty, rightx, righty = self.find_lane_pixels(binary_warped)

        self.fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
        return

    def find_lane_pixels(self, binary_warped):
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0] // 2:, :], axis=0)
        if VISUALIZE:
            # Create an output image to draw on and visualize the result
            out_img = np.dstack((binary_warped, binary_warped, binary_warped))
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0] // 2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        # Set height of windows - based on nwindows above and image shape
        window_height = np.int(binary_warped.shape[0] // self.nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated later for each window in nwindows
        leftx_current = leftx_base
        rightx_current = rightx_base

        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        # Step through the windows one by one
        margin = self.margin
        for window in range(self.nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window + 1) * window_height
            win_y_high = binary_warped.shape[0] - window * window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin

            if VISUALIZE:
                # Draw the windows on the visualization image
                cv2.rectangle(out_img, (win_xleft_low, win_y_low),
                              (win_xleft_high, win_y_high), (0, 255, 0), 2)
                cv2.rectangle(out_img, (win_xright_low, win_y_low),
                              (win_xright_high, win_y_high), (0, 255, 0), 2)

            # Identify the nonzero pixels in x and y within the window #
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                              (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                               (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)

            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > self.minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > self.minpix:
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices (previously was a list of lists of pixels)
        try:
            left_lane_inds = np.concatenate(left_lane_inds)
            right_lane_inds = np.concatenate(right_lane_inds)
        except ValueError:
            # Avoids an error if the above is not implemented fully
            pass

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds]
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]
        self.nonzeroy = nonzeroy
        self.nonzerox = nonzerox
        self.left_lane_inds = left_lane_inds
        self.right_lane_inds = right_lane_inds

        return leftx, lefty, rightx, righty

    def search_around_poly(self, binary_warped):
        # HYPERPARAMETER
        # Choose the width of the margin around the previous polynomial to search
        # The quiz grader expects 100 here, but feel free to tune on your own!
        margin = self.margin
        left_fit = self.left_fit
        right_fit = self.right_fit

        # Grab activated pixels
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])

        ### Set the area of search based on activated x-values ###
        ### within the +/- margin of our polynomial function ###
        self.left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                              left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                              left_fit[1]*nonzeroy + left_fit[2] + margin)))
        self.right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                               right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                               right_fit[1]*nonzeroy + right_fit[2] + margin)))

        # Again, extract left and right line pixel positions
        leftx = nonzerox[self.left_lane_inds]
        lefty = nonzeroy[self.left_lane_inds]
        rightx = nonzerox[self.right_lane_inds]
        righty = nonzeroy[self.right_lane_inds]
        self.nonzeroy = nonzeroy
        self.nonzerox = nonzerox
        return leftx, lefty, rightx, righty

    def fit_poly(self, img_shape, leftx, lefty, rightx, righty):
        # Fit a second order polynomial to each with np.polyfit() ###
        self.left_fit = np.polyfit(lefty, leftx, 2)
        self.right_fit = np.polyfit(righty, rightx, 2)
        return

    def get_poly_plot(self, img_shape):
        # Generate x and y values for plotting
        ploty = np.linspace(0, img_shape[0] - 1, img_shape[0])
        # Calc both polynomials using ploty, left_fit and right_fit ###
        left_fit = self.left_fit
        right_fit = self.right_fit
        left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
        right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]
        return left_fitx, right_fitx, ploty

    def visualize(self, binary_warped):
        left_fitx, right_fitx, ploty = self.get_poly_plot(binary_warped.shape)
        margin = self.margin

        ## Visualization ##
        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((binary_warped, binary_warped, binary_warped)) * 255
        window_img = np.zeros_like(out_img)
        # Color in left and right line pixels
        out_img[self.nonzeroy[self.left_lane_inds], self.nonzerox[self.left_lane_inds]] = [255, 0, 0]
        out_img[self.nonzeroy[self.right_lane_inds], self.nonzerox[self.right_lane_inds]] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx - margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx + margin, ploty])))])

        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx - margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx + margin, ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0, 255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0, 255, 0))

        # Plot the polynomial lines onto the image
        def gen_points(x):
            pts = np.int_(np.dstack((x, ploty)))
            pts = pts.reshape((-1, 1, 2))
            return [pts]
        cv2.polylines(out_img, gen_points(left_fitx), False, (255, 255, 255), thickness=1)
        cv2.polylines(out_img, gen_points(right_fitx), False, (255, 255, 255), thickness=1)
        # plt.plot(left_fitx, ploty, color='yellow')
        # plt.plot(right_fitx, ploty, color='yellow')
        return cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

## Test Bench

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

src = cv2.imread('test_images/test1.jpg')
dst = undistort(src, cam_calib)

BINARIZE_PARAMS = {
    'sobel': {
        'ksize': 25,
        'mag_thr': (5, 100),
        'dir_thr': (0.6, 1.4)  
    },
    'color_thr': (160, 255),
}

binarized = binarize_for_line(dst, BINARIZE_PARAMS)
binary_warped = lane_warper(binarized)
# cv2.imwrite('warped_example.png', binary_warped * 255)

params = {
    'margin': 100,
    'minpix': 50,
    'nwindows': 9,
    'rediscover_thr': 50,
}
lane_finder = LaneFinder(params)
lane_finder.process(binary_warped)
dst = lane_finder.visualize(binary_warped)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(cv2.cvtColor(src, cv2.COLOR_BGR2RGB))
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(cv2.cvtColor(dst, cv2.COLOR_BGR2RGB))
# ax2.imshow(dst, cmap='gray')
ax2.set_title('Processed Image', fontsize=30)